In [1]:
# import pandas as pd
# import math
# import matplotlib.pyplot as plt
import numpy as np

import gurobipy as gp
from gurobipy import GRB, tuplelist

# import pandapower.networks as pn
# import scipy.io

In [2]:
# m = gp.Model('LinMcCormick')
idx_x = np.arange(1,3,1)
set_x = tuplelist([i for i in idx_x])

In [3]:
def define_variables(m):
    # X
    x = m.addVars(set_x, name='x')

    for i in [1,2]:
        x[i].LB, x[i].UB = 0, 1.5

    return x

In [4]:
def define_objective(m,x):
    sum_x = -x[1] + x[1]*x[2] - x[2]

    m.setObjective(sum_x, GRB.MINIMIZE)

In [5]:
def define_constraints(m,x):
    m.addConstr(-6*x[1] + 8*x[2] <= 3)
    m.addConstr(3*x[1] - x[2] <= 3)

In [6]:
m = gp.Model('P1')
x = define_variables(m)
m.update()
define_objective(m,x)
define_constraints(m,x)

m.Params.NonConvex = 2
# m.Params.SolutionLimit = 1
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-25
Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x5c271958
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [3e+00, 3e+00]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -0.7500000
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 8 nonzeros
Presolved model has 1 bilinear constraint(s)
Variable types: 3 continuous, 0 integer (0 binary)
Found he

In [7]:
x

{1: <gurobi.Var x[1] (value 1.1666666666666667)>,
 2: <gurobi.Var x[2] (value 0.5)>}